## Данные

Данные в [архиве](https://drive.google.com/file/d/15o7fdxTgndoy6K-e7g8g1M2-bOOwqZPl/view?usp=sharing). В нём два файла:
- `news_train.txt` тренировочное множество
- `news_test.txt` тренировочное множество

С некоторых новостных сайтов были загружены тексты новостей за период  несколько лет, причем каждая новость принаделжит к какой-то рубрике: `science`, `style`, `culture`, `life`, `economics`, `business`, `travel`, `forces`, `media`, `sport`.

В каждой строке файла содержится метка рубрики, заголовок новостной статьи и сам текст статьи, например:

>    **sport**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею разгромила чехов**&nbsp;&lt;tab&gt;&nbsp;**Сборная Канады по хоккею крупно об...**

# Задача

1. Обработать данные, получив для каждого текста набор токенов
Обработать токены с помощью (один вариант из трех):
    - pymorphy2
    - русского [snowball стеммера](https://www.nltk.org/howto/stem.html)
    - [SentencePiece](https://github.com/google/sentencepiece) или [Huggingface Tokenizers](https://github.com/huggingface/tokenizers)
    
    
2. Обучить word embeddings (fastText, word2vec, gloVe) на тренировочных данных. Можно использовать [gensim](https://radimrehurek.com/gensim/models/word2vec.html) . Продемонстрировать семантические ассоциации. 

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия
    

4.* Реализуйте классификацию с помощью нейросетевых моделей. Например [RuBERT](http://docs.deeppavlov.ai/en/master/features/models/bert.html) или [ELMo](https://rusvectores.org/ru/models/).

In [1]:
import re
import pymorphy2
import pandas as pd
import gensim

In [2]:
lines = list(open('data/news/news_train.txt', 'r', encoding='utf-8'))
#lines = lines2[:15]

In [4]:
def normalized_text(text, morph):
    result = []
    for word in text:
        result.append(morph.parse(word)[0].normal_form)
    return result

def prepare_data(lines): 
    # токенизация
    tokenized_lines = []
    for line in lines:
        label, head, text = line.split('\t')
        head = re.findall(r'\w+', head.lower())
        text = re.findall(r'\w+', text.lower())
        tokenized_lines.append({'label': label, 'head': head, 'text': text})
        
    #приведение к нормальной форме
    morph = pymorphy2.MorphAnalyzer()
    for line in tokenized_lines:
        line['head'] = normalized_text(line['head'], morph)
        line['text'] = normalized_text(line['text'], morph)
    
    return tokenized_lines

train = prepare_data(lines)

In [5]:
train_df = pd.DataFrame(train)
w2v_dim = 150
sentences = train_df['text'].append(train_df['head'])
word2vec = gensim.models.Word2Vec(sentences, size = w2v_dim)

In [6]:
print(word2vec.wv.most_similar(positive=['овечкин']), '\n')
print(word2vec.wv.most_similar(positive=['университет']), '\n')
print(word2vec.wv.most_similar(positive=['samsung']), '\n')

[('малкин', 0.918655514717102), ('испанец', 0.9180492758750916), ('голкипер', 0.9122662544250488), ('кириленко', 0.906489372253418), ('питтсбург', 0.8967603445053101), ('аргентинец', 0.894297182559967), ('надаль', 0.8928858041763306), ('пингвинс', 0.8928146958351135), ('отличиться', 0.8912882804870605), ('дацюк', 0.8901466131210327)] 

[('институт', 0.7764558792114258), ('колледж', 0.7548238039016724), ('мгу', 0.7534153461456299), ('политехнический', 0.7458568811416626), ('факультет', 0.7162067294120789), ('гарвардский', 0.7026298642158508), ('калифорнийский', 0.7022683024406433), ('мичиганский', 0.6985563039779663), ('профессор', 0.6860417127609253), ('стэнфордский', 0.6814392805099487)] 

[('apple', 0.857429027557373), ('lg', 0.8460652828216553), ('htc', 0.8358925580978394), ('смартфон', 0.8341237902641296), ('motorola', 0.8210699558258057), ('планшет', 0.8181561231613159), ('sony', 0.8151872754096985), ('nokia', 0.8067356944084167), ('ipad', 0.8054453134536743), ('microsoft', 0.8039

3. Реализовать алгоритм классификации, посчитать точноть на тестовых данных, подобрать гиперпараметры. Метод векторизации выбрать произвольно - можно использовать $tf-idf$ с понижением размерности (см. scikit-learn), можно использовать обученные на предыдущем шаге векторные представления, можно использовать [предобученные модели](https://rusvectores.org/ru/models/). Имейте ввиду, что простое "усреднение" токенов в тексте скорее всего не даст положительных результатов. Нужно реализовать два алгоритмов из трех:
     - SVM
     - наивный байесовский классификатор
     - логистическая регрессия

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import numpy as np

def document_vectorization(X, word2vec, w2v_dim):
    tfidf = TfidfVectorizer(analyzer=lambda x: x)
    tfidf.fit(X)
    
    tf_weights = { word : tfidf.idf_[ind] for word, ind in tfidf.vocabulary_.items() } # пары {слово : tf-idf }
    
    result = np.array([np.mean([word2vec[w] * tf_weights[w] for w in words if w in word2vec] or [np.zeros(w2v_dim)], axis=0) for words in X])    
    return result

def set_labels(Y):
    labels = {
                'science': 0,
                'style': 1,
                'culture': 2,
                'life': 3,
                'economics': 4,
                'business': 5,
                'travel': 6,
                'forces': 7,
                'media': 8,
                'sport': 9
               }
    return Y.map(lambda label: labels[label]).to_list()
    
x_train = document_vectorization(train_df['text'], word2vec, w2v_dim)
y_train = set_labels(train_df['label'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [8]:
lines_test = list(open('data/news/news_test.txt', 'r', encoding='utf-8'))
test = prepare_data(lines_test)
test_df = pd.DataFrame(test)

In [9]:
x_test = document_vectorization(test_df['text'], word2vec, w2v_dim)
y_test = set_labels(test_df['label'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__contains__` (Method will be removed in 4.0.0, use self.wv.__contains__() instead).
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:17: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [10]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

clf = SVC()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
accuracy_score(y_test, predictions)

0.8503333333333334

In [13]:
from sklearn.naive_bayes import GaussianNB

clf = GaussianNB()
clf.fit(x_train, y_train)
predictions = clf.predict(x_test)
accuracy_score(y_test, predictions)

0.726